데이터 수집 - 스탯티즈  
연도별 WAR 기준 투/타 각각 상위 10명 추출

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

url = 'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=1982&ye=2021&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&pa=0&si=&cn='

In [2]:
r = requests.get(url).text
soup = BeautifulSoup(r, 'lxml')

In [6]:
soup.select('tr')[2]

<tr><td style="white-space:nowrap;text-align:center;vertical-align:middle;">1</td><td style="white-space:nowrap;text-align:center;vertical-align:middle;"><a href="player.php?name=%EC%9D%B4%EC%A2%85%EB%B2%94&amp;birth=1970-08-15">이종범</a></td><td style="white-space:nowrap;text-align:center;vertical-align:middle;"><span style="font-size:11px;padding-left:3px;padding-right:3px;"><span style="display:inline-block;width:15px;background:#DB0000;color:white;">94</span><span style="display:inline-block;width:15px;background:#DB0000;color:white;">해</span><span style="display:inline-block;width:20px;">SS</span></span></td><td style="text-align:right;border-left:2px solid #333333;background:#DDDDDD;color:white;font-weight:bold;border-left:2px solid #333333; border-right:2px solid #333333;"><font style="color:#000000;"> <span style="padding-left:5px;padding-right:5px;">11.77</span> </font></td><td style="text-align:right;"><font style="color:#000000;"> <span style="padding-left:5px;padding-right:5p

In [7]:
soup.select('tr')[2].text

'1이종범94해SS 11.77  124  561  499  113  196  27  5  19  290  77  84  15  51  6  14  31  2  1  4  .393  .452  .581  1.033  .462  198.3  11.77 '

문제점 : 삼미-삼성, SK-SSG, KT-KIA의 팀 첫자가 동일하기 때문에 글자로만 팀을 구분하기 힘듬. 따라서 스탯티즈에서 팀을 구분하는데 쓰고 있는 색을 통해 팀을 구분해야함.

In [111]:
soup.select('tr')[2].select('span')[2]['style']

'display:inline-block;width:15px;background:#DB0000;color:white;'

In [13]:
for context in soup.select('tr')[2].select('td'):
    print(context.text, end=' ')

1 이종범 94해SS  11.77   124   561   499   113   196   27   5   19   290   77   84   15   51   6   14   31   2   1   4   .393   .452   .581   1.033   .462   198.3   11.77   

In [35]:
isend = False
for a in soup.select('tr'):
    if len(a.select('td')) == 0:
        continue
    for context in a.select('td'):
        if '/' in context.text:
            isend = True
            break
        print(context.text, end=' ')
    print()
    if isend:
        break

1 이종범 94해SS  11.77   124   561   499   113   196   27   5   19   290   77   84   15   51   6   14   31   2   1   4   .393   .452   .581   1.033   .462   198.3   11.77   
2 테임즈 15N1B  10.71   142   595   472   130   180   42   5   47   373   140   40   8   103   13   11   91   7   0   7   .381   .498   .790   1.288   .530   222.3   10.71   
3 심정수 03현RF  10.19   133   601   460   110   154   16   1   53   331   142   6   2   124   9   17   63   14   0   8   .335   .478   .720   1.197   .498   210.7   10.19   
4 이종범 97해SS  9.70   125   577   484   112   157   28   3   30   281   74   64   15   87   3   30   49   9   0   3   .324   .428   .581   1.009   .431   173.2   9.70   
5 이종범 96해SS  9.52   113   525   449   94   149   28   1   25   254   76   57   12   67   7   15   39   4   0   2   .332   .425   .566   .991   .440   184.6   9.52   
6 이승엽 02삼1B  8.96   133   617   511   123   165   42   2   47   352   126   1   2   89   15   5   109   11   0   2   .323   .436   .689   1.125   .472   

In [167]:
def get_team(context):
    team = context.text.strip()[2]
    style = context.select('span')[2]['style']
    
    if team == '두':
        return '두산'
    elif team == 'N':
        return 'NC'
    elif team == '키':
        return '키움'
    elif team == '한':
        return '한화'
    elif team == 'L':
        return 'LG'
    elif team == '롯':
        return '롯데'
    elif team == '해':
        return '해태'
    elif team == '현':
        return '현대'
    elif team == '청':
        return '청보'
    elif team == 'M':
        return 'MBC'
    elif team == 'O':
        return 'OB'
    elif team == '태':
        return '태평양'
    elif team == '빙':
        return '빙그레'
    elif team == '쌍':
        return '쌍방울'
    elif team == '넥':
        return '넥센'
    elif team == '히':
        return '히어로즈'
    elif team == 'k':
        return 'KT'
    elif team == 'K':
        if 'background:#C90000' in style:
            return 'KIA'
        else:
            return 'KT'
    elif team == '삼':
        if 'background:#001EC9' in style:
            return '삼성'
        else:
            return '삼미'
    elif team == 'S':
        if 'background:#FF1212' in style:
            return 'SK'
        else:
            return 'SSG'

In [137]:
data = {}
stats = ['이름', '연도', '소속', 'WAR', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비',
         '타율', '출루', '장타', 'OPS', 'wOBA', 'wRC+']
for stat in stats:
    data[stat] = []
for i, a in enumerate(soup.select('tr')):
    if i == 12:
        break
    
    for j, context in enumerate(a.select('td')):
        if j == 0 or j > len(stats)-1:
            continue
        
        elif j == 1:
            data[stats[j-1]].append(context.text.strip())
        
        elif j == 2:
            team = get_team(context)
            data[stats[j]].append(team)
            data[stats[j-1]].append('')

        else:
            data[stats[j]].append(context.text.strip())

In [138]:
df = pd.DataFrame(data)
df

,이름,연도,소속,WAR,G,타석,타수,득점,안타,2타,...,삼진,병살,희타,희비,타율,출루,장타,OPS,wOBA,wRC+
0,이종범,,해태,11.77,124,561,499,113,196,27,...,31,2,1,4,.393,.452,.581,1.033,.462,198.3
1,테임즈,,NC,10.71,142,595,472,130,180,42,...,91,7,0,7,.381,.498,.790,1.288,.530,222.3
2,심정수,,현대,10.19,133,601,460,110,154,16,...,63,14,0,8,.335,.478,.720,1.197,.498,210.7
3,이종범,,해태,9.70,125,577,484,112,157,28,...,49,9,0,3,.324,.428,.581,1.009,.431,173.2
4,이종범,,해태,9.52,113,525,449,94,149,28,...,39,4,0,2,.332,.425,.566,.991,.440,184.6
5,이승엽,,삼성,8.96,133,617,511,123,165,42,...,109,11,0,2,.323,.436,.689,1.125,.472,198.5
6,이승엽,,삼성,8.91,132,614,486,128,157,33,...,114,7,0,4,.323,.458,.733,1.190,.486,191.8
7,이대호,,롯데,8.82,127,552,478,99,174,13,...,77,15,0,3,.364,.444,.667,1.111,.482,192.4
8,장종훈,,빙그레,8.79,126,558,464,104,160,24,...,88,12,0,3,.345,.450,.640,1.090,.482,204.2
9,홍현우,,해태,8.75,126,549,443,83,147,30,...,64,8,1,4,.332,.453,.533,.985,.442,186.1


In [145]:
data = {}
stats = ['이름', '연도', '소속', 'WAR', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비',
         '타율', '출루', '장타', 'OPS', 'wOBA', 'wRC+']
for stat in stats:
    data[stat] = []

In [146]:
def batter(year):
    global data
    url = f'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&pa=0&si=&cn='
    r = requests.get(url).text
    soup = BeautifulSoup(r, 'lxml')
    
    for i, a in enumerate(soup.select('tr')):
        if i == 12:
            break

        for j, context in enumerate(a.select('td')):
            if j == 0 or j > len(stats)-1:
                continue

            elif j == 1:
                data[stats[j-1]].append(context.text.strip())

            elif j == 2:
                team = get_team(context)
                data[stats[j]].append(team)
                data[stats[j-1]].append(year)

            else:
                data[stats[j]].append(context.text.strip())

In [147]:
for year in range(1982, 2022):
    batter(year)

In [150]:
bat = pd.DataFrame(data)
bat

,이름,연도,소속,WAR,G,타석,타수,득점,안타,2타,...,삼진,병살,희타,희비,타율,출루,장타,OPS,wOBA,wRC+
0,백인천,1982,MBC,5.88,72,298,250,55,103,23,...,17,5,0,3,.412,.497,.740,1.237,.528,227.0
1,윤동균,1982,OB,4.13,77,334,284,54,97,20,...,27,3,0,4,.342,.428,.532,.960,.430,164.6
2,이만수,1982,삼성,4.12,78,322,270,46,78,16,...,27,13,0,1,.289,.399,.500,.899,.405,147.6
3,김봉연,1982,해태,3.68,74,304,269,55,89,14,...,16,8,0,1,.331,.405,.636,1.040,.453,180.5
4,김우열,1982,OB,3.46,62,255,210,43,65,8,...,25,0,0,1,.310,.428,.533,.961,.426,161.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,구자욱,2021,삼성,5.01,139,610,543,107,166,30,...,98,9,1,12,.306,.361,.519,.881,.394,134.0
396,전준우,2021,롯데,4.93,144,619,552,88,192,46,...,71,10,1,8,.348,.405,.469,.874,.406,140.4
397,알테어,2021,NC,4.91,143,565,492,83,134,19,...,156,6,0,5,.272,.358,.514,.872,.393,132.0
398,김재환,2021,두산,4.77,137,566,475,86,130,23,...,127,9,0,5,.274,.382,.501,.883,.401,148.2


In [151]:
bat.isna().sum()

이름      0
연도      0
소속      0
WAR     0
G       0
타석      0
타수      0
득점      0
안타      0
2타      0
3타      0
홈런      0
루타      0
타점      0
도루      0
도실      0
볼넷      0
사구      0
고4      0
삼진      0
병살      0
희타      0
희비      0
타율      0
출루      0
장타      0
OPS     0
wOBA    0
wRC+    0
dtype: int64

In [155]:
bat.to_csv('./data/batter.csv', encoding='utf-8-sig', index=False)

In [168]:
data = {}
stats = ['이름', '연도', '소속', 'WAR', '출장', '완투', '완봉', '선발', '승', '패', '세', '홀', '이닝', '실점', '자책', '타자', '피안', '피2', '피3', '피홈', '피볼', '고4', '사구', '삼진', '보크', '폭투',
         'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+']
for stat in stats:
    data[stat] = []

In [169]:
def pitcher(year):
    global data
    url = f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
    r = requests.get(url).text
    soup = BeautifulSoup(r, 'lxml')
    
    for i, a in enumerate(soup.select('tr')):
        if i == 12:
            break

        for j, context in enumerate(a.select('td')):
            if j == 0 or j > len(stats)-1:
                continue

            elif j == 1:
                data[stats[j-1]].append(context.text.strip())

            elif j == 2:
                team = get_team(context)
                data[stats[j]].append(team)
                data[stats[j-1]].append(year)

            else:
                data[stats[j]].append(context.text.strip())

In [170]:
for year in range(1982, 2022):
    pitcher(year)

In [171]:
pit = pd.DataFrame(data)
pit

,이름,연도,소속,WAR,출장,완투,완봉,선발,승,패,...,고4,사구,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+
0,박철순,1982,OB,9.54,36,15,2,19,24,4,...,1,6,108,0,4,1.84,2.82,0.97,211.2,137.8
1,황규봉,1982,삼성,8.62,47,8,2,16,15,11,...,3,6,99,1,4,2.47,2.67,1.00,160.6,148.5
2,노상수,1982,롯데,7.05,44,12,2,20,14,19,...,6,10,141,2,4,2.94,3.03,1.18,134.9,131.3
3,하기룡,1982,MBC,6.78,43,7,4,16,13,10,...,4,13,107,2,6,2.30,2.95,1.11,172.3,134.4
4,권영호,1982,삼성,5.12,32,6,3,21,15,5,...,0,7,72,1,1,2.37,3.91,1.07,167.5,101.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,요키시,2021,키움,4.66,31,0,0,31,16,9,...,2,4,131,0,9,2.93,3.54,1.20,151.4,122.4
396,뷰캐넌,2021,삼성,4.55,30,1,1,30,16,5,...,2,2,162,0,7,3.10,3.46,1.31,145.5,130.3
397,루친스키,2021,NC,4.48,30,0,0,30,15,10,...,0,10,177,2,6,3.17,3.32,1.20,143.1,135.9
398,수아레즈,2021,LG,4.43,23,0,0,22,10,2,...,0,4,126,0,2,2.18,2.77,1.15,197.4,157.7


In [172]:
pit.isna().sum()

이름      0
연도      0
소속      0
WAR     0
출장      0
완투      0
완봉      0
선발      0
승       0
패       0
세       0
홀       0
이닝      0
실점      0
자책      0
타자      0
피안      0
피2      0
피3      0
피홈      0
피볼      0
고4      0
사구      0
삼진      0
보크      0
폭투      0
ERA     0
FIP     0
WHIP    0
ERA+    0
FIP+    0
dtype: int64

In [173]:
pit.to_csv('./data/pitcher.csv', encoding='utf-8-sig', index=False)